<p align="center">
    <img src="https://drive.google.com/uc?id=1Wf35FM2aFklfwq7t_UEEd1pxZlIXvfZ3
" width="1200"/>
</p>


# 개요

1. 아래 링크의 Ortools를 활용한 해법을 참고하였습니다.(2위 팀 공개 코드)

   https://www.kaggle.com/code/spacelx/2020-hc-dd-2nd-place-solution-w-or-tools

2. 이 해법은 "OR Tools를 이용해서 최적 경로를 구하는 부분" 과 "인간의 직관을 활용하여 배치를 나누는 부분"의 2가지 핵심 사항으로 구성되어 있습니다.

3. 저는 이 과제에서 아래의 목록을 수행했습니다.

   - EDA와 평가 지표 분석을 통해 높은 점수를 받기 위해서는 어떻게 접근해야 하는지 스스로 분석
     (해법에는 어떻게 풀었다는 것인지에 대한 내용 위주로 나와있고, 지표를 통한 구체적인 분석 내용이 없다.)
   - 분석 결과를 토대로 기존 풀이의 풀이 전략을 분석하여 풀이의 **개선 방안**을 적용 (Leader Board Score 기존 방식보다 높은 값을 산출하기 성공)

     (단, 실제 Leader Board Score가 아닌, 2위 팀 공개 코드 노트북을 그대로 다시 제출했을 때 나오는 점수 대비 성능 개선이 이루어졌습니다.)

   - 전체 코드를 **다시 처음부터 직접 구현하여** ortools을 사용한 최적 경로를 찾는 방법을 익힘

4. 제가 성능 개선을 위해 기존 풀이에서 바꾼 포인트는 **드론 배차 시 Weight \* distance가 아닌 새로운 Priority Metric을 사용한 점** 이며
   **⭐️ 이모티콘**과 함께 아래에 표현해두었습니다.


# 문제 요약

> 드론을 이용하여 Warehouse에 있는 아이템을 고객에게 최소 시간으로 배송하고자 한다.

## Input

- 드론의 개수, 드론의 최대 배송 용량,
- 창고의 개수, 각 창고에 있는 재고의 개수, 창고의 위치
- 상품 종류의 개수, 각 상품별 무게
- 주문 발생 위치, 주문별 요구 아이템 및 개수
- 최대 limit turn인 $T$가 주어진다. (즉 전체 배송을 완료한 시간이 T를 넘으면 안 된다.)

## Turn 의 개념

- 드론은 Turn 시간 단위로 이동하며, 각 동작별 Turn 소요 개수는 아래와 같다.
  - 같은 장소에서 한 가지 제품군에 대하여 LOAD, UNLOAD, DELIVER (개수는 무관) => 1 Turn
  - 이동 => ceil(두 좌표의 유클리디안 거리) Turns
- Output으로 각 드론에 대해서 LOAD, UNLOAD, STOP, DELIVER 명령을 부여한 txt 파일을 제출한다.

## 평가 지표

- $n$번 Order가 완전히 배송 완료 되는 시점의 turn을 $t_n$이라고 할 때 평가 지표는 아래와 같다.
  $
  100\times\sum_{n}\frac{T-t_n}{T}
  $


# 평가 지표 분석

### 작업 순서의 영향을 받으며, 빨리 처리되는 Order가 먼저 처리되어야 점수가 높다

- 거리가 10인 지점의 주문과 거리가 2인 지점의 주문을 처리하는 경우를 생각해보자.
- T = 20 이라고 가정하자.
- 단순 Scaler인 100을 곱하는 과정은 생략하자.
- 이 때 거리가 10인 지점 방문 후 2인 지점을 방문하면
  $$
  \frac {20 - 10} {20} + \frac{20 - 12} {20} = 2 - \frac{10}{20} - \frac{12}{20}
  $$
- 이번엔 거리가 2인 지점 방문 후 10인 지점을 방문하면

  $$
  \frac {20 - 2} {20} + \frac{20 - 12} {20} = 2 - \frac{2}{20} - \frac{12}{20}
  $$

- 즉 평가지표는 합산하는 방식처럼 보이지만, 사실은 총점에서 감점하는 방식을 취하고 있다. 처리해야하는 order의 수 \* 100 이 총점이며 여기에서 걸린 시간이 적을 수록 적게 감점하고 있는 형태인 것이다.
- 위에 비교한 사례처럼 "빨리 끝나는 주문을 먼저 처리해야" 감점되는 양이 훨씬 작다는 것을 알 수 있다.
- 따라서 Order와 Warehouse 사이의 거리가 가까울수록 유리하다.
- 또한 Order에 총 Weight가 작을수록 여러개의 드론이 병렬적으로 배송하거나, 한 번의 이동만으로 Order 처리를 끝낼 수 있다. (Order는 무조건 전부다 처리가 되어야만 완료 처리가 된다.)


## 이제 EDA를 통해 데이터의 특성에 대해 알아보고, 풀이 전략을 수립하자


## Import Modules


In [ ]:
%pip install - -upgrade ortools


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.graph.python import min_cost_flow
from collections import Counter
from IPython.display import display
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as mcm
import matplotlib as mpl
import pickle


## Load Data From Input File


In [ ]:
with open("/kaggle/input/hashcode-drone-delivery/busy_day.in") as f:
    lines = f.readlines()
lines = [list(map(int, line.rstrip().split())) for line in lines]


In [ ]:
ROWS, COLS, DRONE_NUM, TURN_NUM, PAYLOAD = lines[0]
PRODUCT_TYPE_LEN = lines[1][0]
PRODUCT_WEIGHTS = lines[2]
WAREHOUSE_LEN = lines[3][0]
warehouse_info_lines = lines[4:4 + (2 * WAREHOUSE_LEN)]
WAREHOUSE_INFO = []
for i in range(WAREHOUSE_LEN):
    warehouse = {"id": i,
                 "pos_row": warehouse_info_lines[2*i][0],
                 "pos_col": warehouse_info_lines[2*i][1],
                 "stock_list": warehouse_info_lines[2*i + 1]}
    WAREHOUSE_INFO.append(warehouse)
ORDERS_LEN = lines[4 + (2 * WAREHOUSE_LEN)][0]
order_info_lines = lines[-(3 * ORDERS_LEN):]
ORDERS_INFO = []
for i in range(ORDERS_LEN):
    order = {"id": i,
             "pos_row": order_info_lines[3*i][0],
             "pos_col": order_info_lines[3*i][1],
             "ordered_items": order_info_lines[3*i + 2],
             }
    ORDERS_INFO.append(order)
PAYLOAD

## Make DataFrames

- **Entities** : product_df, warehouse_df, order_df

### 1. product_df

- fields : id, weight, order_id_list, stock_warehouse_list

### 2. warehouse_df

- fields : id, pos_row, pos_col, stock_list

### 3. orders_df

- fields : id, pos_row, pos_col, ordered_items


In [ ]:
product_df = pd.DataFrame([{"id": idx, "weight": weight, "order_id_list": [], "stock_warehouse_list": []} for idx, weight in enumerate(
    PRODUCT_WEIGHTS)], index=None).set_index("id", inplace=False)
for order in ORDERS_INFO:
    for item in order["ordered_items"]:
        product_df.loc[item, "order_id_list"].append(order["id"])
for warehouse in WAREHOUSE_INFO:
    for item, stock in enumerate(warehouse["stock_list"]):
        if stock > 0:
            product_df.loc[item, "stock_warehouse_list"].append(
                (warehouse["id"], stock))
warehouse_df = pd.DataFrame(WAREHOUSE_INFO).set_index("id", inplace=False)
orders_df = pd.DataFrame(ORDERS_INFO).set_index("id", inplace=False)
display(product_df)
display(warehouse_df)
display(orders_df)


## EDA


### Warehouses and Orders

우선 Warehouse와 Orders의 위치를 파악한다.


In [ ]:
cmap = mcm.get_cmap("nipy_spectral")
wh_colors = [cmap(i) for i in np.linspace(0, 1, num=WAREHOUSE_LEN)]

fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(warehouse_df["pos_col"], warehouse_df["pos_row"],
           ec="black", color=wh_colors, s=100, zorder=1)
ax.scatter(orders_df["pos_col"], orders_df["pos_row"],
           color="black", s=2, zorder=0)
ax.set_xlabel("col")
ax.set_ylabel("row")
ax.axis("equal")

norm = mpl.colors.Normalize(vmin=0, vmax=WAREHOUSE_LEN-1)
cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
cb.set_label('Warehouse Index')

plt.show()

- 1번과 8번 Warehouse가 주문 밀집지와 멀리 떨어져있다.
- 3번과 5번 Warehouse가 주문 밀집지와 가까운 위치에 존재한다.
- 3번과 5번 Warehouse는 주문 밀집지와 가까우므로 빨리 처리될 수 있는 Order가 많을 것이라고 예측할 수 있다.


### Weight Distribution

아이템들의 무게 분포를 살펴보자


In [ ]:
design_setting = {"color": "#29BCB7", "ec": "white"}
weights = product_df.loc[:, "weight"]
sns.displot(weights, kde=True, **design_setting)
plt.show()


- 약간 좌로 치우친 정규분포 형태를 가지고 있다.
- PAYLOAD가 200이므로 드론은 한 번 배송에서 평균적으로 4~5개 정도의 짐을 실을 수 있다.
- 그러나 극단적으로는 15~16개를 담을 수 있는 경우도 있고, 1개 밖에 못 담는 경우도 존재한다.


### Order 별 weight의 크기와 WareHouse 재고의 Weight 크기

각 Warehouse가 담고 있는 Weight와 Order가 가지고 있는 Weight의 크기를 조사해보자.


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

orders_df['total_weight'] = orders_df['ordered_items'].apply(
    lambda x: sum(map(lambda y: product_df.loc[y, 'weight'], x)))
warehouse_total_weight = []
for stock_list in warehouse_df.loc[:, 'stock_list']:
    result = 0
    for idx, stock in enumerate(stock_list):
        result += (product_df.loc[idx, "weight"] * stock)
    warehouse_total_weight.append(result)

warehouse_df['total_weight'] = warehouse_total_weight

scatter1 = sns.scatterplot(data=warehouse_df, x='pos_col', y='pos_row',
                           hue='total_weight', ec='black', s=500, ax=ax, zorder=1)

scatter2 = sns.scatterplot(data=orders_df, x='pos_col', y='pos_row',
                           hue='total_weight', size='total_weight', sizes=(10, 200),
                           ax=ax, zorder=0, palette="Blues")

scatter1.legend_.set_title("")

ax.set_xlabel('col')
ax.set_ylabel('row')
ax.axis('equal')

plt.show()

- 가장 중심에 있는 Warehouse와 중심과 바깥에 있는 Warehouse에 가장 많은 Weight이 있다.
- "가장 바깥에 있는 Warehouse에 배송 물류가 많은 것"은 배송 시간을 늘릴 수 있는 여지가 있는 위험한 포인트이다.
- 주문별 Weight의 경우 거의 랜덤 분포나 다름없이 특별한 규칙성을 보이지 않는다.


### Order 별 Weight의 크기와 Warehouse에 있는 stock의 개수

- 이번엔 Warehouse의 Stock의 개수 관점에서 보자.


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

warehouse_df['total_stocks'] = warehouse_df['stock_list'].apply(
    lambda x: sum(x))

scatter1 = sns.scatterplot(data=warehouse_df, x='pos_col', y='pos_row',
                           hue='total_stocks', ec='black', s=500, ax=ax, zorder=1)

scatter2 = sns.scatterplot(data=orders_df, x='pos_col', y='pos_row',
                           hue='total_weight', size='total_weight', sizes=(10, 200),
                           ax=ax, zorder=0, palette="Blues")

scatter1.legend_.set_title("")

ax.set_xlabel('col')
ax.set_ylabel('row')
ax.axis('equal')

plt.show()

- 전반적으로 위에서 본 결과와 비슷하다.
- 따라서 Warehouse가 가지고 있는 item의 Density는 모두 비슷하다고 여겨진다. 즉, 특정 Warehouse에 무거운 item이 편중되어있지 않다.


### Warehouse_Density

실제로 확인해보자.


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

warehouse_df['density'] = warehouse_df['total_weight'] / \
    warehouse_df['total_stocks']

scatter1 = sns.scatterplot(data=warehouse_df, x='pos_col', y='pos_row',
                           hue='density', ec='black', s=500, ax=ax, zorder=1)

scatter2 = sns.scatterplot(data=orders_df, x='pos_col', y='pos_row',
                           hue='total_weight', size='total_weight', sizes=(10, 200),
                           ax=ax, zorder=0, palette="Blues")

scatter1.legend_.set_title("")

ax.set_xlabel('col')
ax.set_ylabel('row')
ax.axis('equal')


plt.show()

- 예상한 바와 같이 숫자 폭을 보면 Density가 크게 차이 나지 않는 모습을 보인다.
- 그래도 그 중에서 중심지로부터 먼 창고가 더 높은 Density를 가지므로 해당 창고가 Weight와 Distance를 모두 높여 점수를 크게 하락시킬 수 있는 요인을 가지고 있다.
- 따라서 Weight가 다른 창고에 비해 큰 것이 맞다면 해당 Warehouse의 작업 배치를 후반부로 두는 방안을 고려해볼 수 있다.
- 한 번 자세히 직접 비교해보자.


## Comparison between Far Warehouses and Nearby Warehouses

아이템들의 무게를 비교해보자


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
weights = []
design_setting_far = {"color": "#29BCB7", "ec": "white"}
design_setting_nearby = {"color": "#E89887", "ec": "white"}

for stock in set(warehouse_df.loc[1, "stock_list"]):
    weights.append(product_df.loc[stock, "weight"])
sns.histplot(weights, kde=True, **design_setting_far, ax=axes[0][0])

for stock in set(warehouse_df.loc[8, "stock_list"]):
    weights.append(product_df.loc[stock, "weight"])
sns.histplot(weights, kde=True, **design_setting_far, ax=axes[0][1])

for stock in set(warehouse_df.loc[3, "stock_list"]):
    weights.append(product_df.loc[stock, "weight"])
sns.histplot(weights, kde=True, **design_setting_nearby, ax=axes[1][0])

for stock in set(warehouse_df.loc[5, "stock_list"]):
    weights.append(product_df.loc[stock, "weight"])
sns.histplot(weights, kde=True, **design_setting_nearby, ax=axes[1][1])

axes[0][0].set_title("warehouse 1 (Far)")
axes[0][1].set_title("warehouse 8 (Far)")
axes[1][0].set_title("warehouse 3 (NearBy)")
axes[1][1].set_title("warehouse 5 (NearBy)")
fig.tight_layout()
plt.show()


- 중심에 있는 Warehouse가 item의 개수가 더 많다.
- 분포를 보면 Warehouse 3, 5 역시 무게 100의 비율이 꽤 많은 편이므로 Warehouse 1, 8이라고 해서 나중에 처리하거나 할 필요는 없을 것을 보인다.
- Warehouse 3, 5를 배송할 때 distance가 작다면 무게 100짜리 비율이 크기 때문에 드론에 기껏해야 하나를 담을 수 있는 경우가 많아지므로 이는 치명적이다.
- 따라서 weight, distance를 모두 잘 고려해야 할 필요가 있다.


# How To Solve It

문제 풀이의 과정은 총 3개의 Part로 나누어지게 된다.
이에 앞서 준비단계가 있다.

모든 주문을 한 개의 아이템 단위로 나누어 생각한다. 즉, (주문 번호, 아이템 종류, item_id) 형태로 나타낸다.
여기서 item_id란 예를 들어 300번 주문에 20번 아이템이 2개 주문이 있을 경우 각 주문은 (300, 20, 0), (300, 20, 1) 이렇게 아이템 단위로 주문을 생각하겠다는 것이다.
이를 Order Unit 이라고 하자.

이제 문제 풀이의 3개의 파트를 알아보자.

## 1. Optimize Product Distribution

- 각 Order Unit을 몇 번 Warehouse에서 가져와야 하는지 구한다.

---

## 2. Create Delivery Unit Route

- Warehouse에서 짐을 싣고 -> 실은 짐을 모든 Order Unit에게 나누어주고 -> 다음 Warehouse로 이동 루틴을 탄다고 가정하자.
- Warehouse에서 짐을 싣고 -> 실은 짐을 모든 Order Unit에게 나누어 주는 과정을 1개의 Unit Route라고 할 때
- 최단 Distance로 모든 Order Unit을 처리할 수 있는 Unit Route 들의 집합을 구한다.
- 이 때 빨리 끝나는 Order unit (특히, 한 개의 Order로 묶여있는 Unit Set) 을 가능한 빨리 처리하도록 구한다.

---

## 3. Scheduling

- 각 Unit Route 들에 대하여 담당 Drone을 할당한다.
- Distance, Weight를 고려하였을 때 빨리 끝나는 Order unit (특히, 한 개의 Order로 묶여있는 Unit Set) 을 가능한 빨리 처리하도록 구한다.


# 1. Optimize Product Distribution

### Problem

- 각 주문에 대해, 각 물건을 어떤 창고로부터 가져올 것인가?

### Solution

- 각 물건에 대해 창고를 Source, 주문 위치를 Sink 라고 두고 Min-Cost-Flow 문제를 해결한다.
- 그러면 각 주문에 대해 각 물건이 어떤 source에서 나오는지를 알 수 있다.

### Output

- from_wh_df 라는 3d Numpy 변수에 결과를 담는다.
- from_wh_df[order_id][item_id] = [어디 창고에서 왔는지]
- Ex : from_wh_df[2][6] = [1, 3, -1]

  - 2번 주문에서 6번 물건이 2개 주문되었는데, 각각은 1번, 3번 창고에서 오는 것이 최적화되어 있다.
  - -1은 해당 item 주문이 없음을 의미


### from_wh_df 결과 산출


In [ ]:
max_item_num = max(orders_df.loc[:, "ordered_items"].apply(
    lambda x: max(Counter(x).values())))
from_wh_df = np.full((len(orders_df), len(product_df), max_item_num), -1)
for product in range(len(product_df)):
    # Define Sources (Warehouse)
    sources = warehouse_df.copy()
    sources["stock"] = sources["stock_list"].apply(lambda x: x[product])
    sources = sources[sources["stock"] > 0]

    # Define Sinks (Orders)
    sinks = orders_df[orders_df["ordered_items"].apply(lambda x: product in x)]

    # Initialize Simple Min Cost Flow
    smcf = min_cost_flow.SimpleMinCostFlow()

    # Add All Arcs
    for i in sources.index:
        for j in sinks.index:
            dist = np.ceil(np.sqrt(
                (sources.loc[i, "pos_row"] - sinks.loc[j, "pos_row"]) ** 2 +
                (sources.loc[i, "pos_col"] - sinks.loc[j, "pos_col"]) ** 2
            ))
            capacities = sinks.loc[j, "ordered_items"].count(product)
            smcf.add_arcs_with_capacity_and_unit_cost(
                i, j+WAREHOUSE_LEN, int(capacities), int(dist)
            )
        # Add Overflow Node
        smcf.add_arcs_with_capacity_and_unit_cost(
            i, int(1e4), int(1e4), int(1e4)
        )

    # Add Supplies
    # Supplies
    for i in sources.index:
        smcf.set_nodes_supplies(i, int(sources.loc[i, "stock"]))
    # Demands
    for j in sinks.index:
        smcf.set_nodes_supplies(
            j+WAREHOUSE_LEN, -int(sinks.loc[j, "ordered_items"].count(product)))
    # Overflow Node
    product_sum = sinks.loc[:, "ordered_items"].apply(
        lambda x: x.count(product)).sum()
    stocks_sum = sources.loc[:, "stock"].sum()

    smcf.set_nodes_supplies(
        int(1e4), int(product_sum - stocks_sum)
    )

    if smcf.solve() == smcf.OPTIMAL:
        for arc in range(smcf.num_arcs()):
            if not smcf.flow(arc):
                continue
            if smcf.unit_cost(arc) == int(1e4):
                continue

            this_order = smcf.head(arc) - WAREHOUSE_LEN
            this_wh = int(smcf.tail(arc))
            this_flow = smcf.flow(arc)

            for each_item in range(from_wh_df.shape[-1]):
                if not this_flow:
                    break
                if from_wh_df[this_order, product, each_item] == -1:
                    from_wh_df[this_order, product, each_item] = this_wh
                    this_flow -= 1

    else:
        raise Exception(
            f"product_id {product}: distribution could not be optimized")


### Visualize warehouse 5 (Nearby), warehouse 1 (Far) Results


In [ ]:
cmap = mcm.get_cmap("nipy_spectral")
wh_colors = [cmap(i) for i in np.linspace(0, 1, num=WAREHOUSE_LEN)]

fig, ax = plt.subplots(figsize=(20, 8), nrows=1, ncols=2)
norm = mpl.colors.Normalize(vmin=0, vmax=WAREHOUSE_LEN-1)
cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
cb.set_label('Warehouse Index')

for i, wh in enumerate([5, 1]):
    ax[i].scatter(warehouse_df["pos_col"], warehouse_df["pos_row"],
                  ec="black", color=wh_colors, s=100, zorder=1)
    ax[i].scatter(orders_df["pos_col"], orders_df["pos_row"],
                  color="black", s=2, zorder=0)
    ax[i].set_xlabel("col")
    ax[i].set_ylabel("row")
    ax[i].axis("equal")

    idx = np.where(from_wh_df == wh)
    for order in idx[0]:
        ax[i].plot([orders_df.loc[order, "pos_col"], warehouse_df.loc[wh, "pos_col"]],
                   [orders_df.loc[order, "pos_row"],
                       warehouse_df.loc[wh, "pos_row"]],
                   color="#29BCB7", lw=0.5)

plt.show()

# 2. Create Delivery Unit Route

### Problem

- WareHouse에서 한 번에 짐을 실어서 배송을 수행하려고 한다.
- 이 때 한 번에 짐을 실어서 배송을 수행하는 것을 하나의 Unit Route라고 하자.
- 우리는 이후에 각 드론들에게 Unit Route를 하나씩 부여하는 방식으로 작업을 수행할 것이다.
- 이 때 distance의 총합을 최소화 하는 Unit Route 집합을 구성하시오.
- 현재 각 주문의 각 상품에 대하여 몇 번 Warehouse에서 가져와야 하는지는 결정되어있다.
- distance 뿐 아니라, weight의 상황도 고려하여 문제를 해결해야한다.

### Solution

- 각 Order 들에 대해서 CVRP 문제를 푸는데, 최대 가용 Vehicle의 개수를 Order의 개수와 똑같이 맞춘다.
- 이렇게 하면 창고로 되돌아가서 다른 루트로 가는 루트가 생기지 않게 된다.
- 그리고, 이미 어떤 Warehouse에서 어떤 order로 어떤 제품이 가야하는지는 값이 구해져 있으니, 같은 Warehouse에서 처리해야 하는 주문들과 해당 warehouse에 대한 distance metrics를 입력으로 넣는다.
- 이 때 경로 계산 시, 한 번에 모든 계산을 처리하는 방식과 특정 Batch로 나누어서 처리하는 방식이 있다.
- 특정 Batch로 나누어서 처리하면 우선순위가 높은 order들끼리 먼저 최적 경로를 계산한 뒤, 우선 순위가 낮은 order의 최적 경로를 계산한다.
- 그래서 우선 순위가 높은 order를 더 빠르게 처리가 가능하고, 이는 위에 기술한 바와 같이 평가 점수를 올릴 수 있다.
- 우선순위는 Weight \* Distance 값을 이용해, 이 값이 클수록 배송 난이도가 어려운 것으로 판단하여 난이도가 쉬운 것부터 처리한다. (그래야 짧은 경로를 더 빨리 처리할 수 있다.)

### Output

- routes_df라는 df에 산출된 unit route 집합의 원소를 담는다.
- routes_df에는 시작 warehouse, 해당 route의 command, 종점의 위치, 경로의 거리를 담는다.


### Weight \* Distance 값 산출


In [ ]:
weight_dist = np.zeros((len(orders_df)), dtype=int)
accumulated_weight_dist = np.zeros((len(orders_df)), dtype=int)

for wh in warehouse_df.index:
    filtered_index = np.where(from_wh_df == wh)
    weight_this_wh = np.zeros((len(orders_df)), dtype=int)
    for i in range(len(filtered_index[0])):
        weight_this_wh[filtered_index[0][i]
                       ] += product_df.loc[filtered_index[1][i], "weight"]
    weight_dist += np.ceil(np.sqrt(
        (orders_df["pos_row"] - warehouse_df.loc[wh, "pos_row"]) ** 2 +
        (orders_df["pos_col"] - warehouse_df.loc[wh, "pos_col"]) ** 2
    )).astype(int) * weight_this_wh
    orders_df[f"weight_dist_{wh}"] = weight_dist
    accumulated_weight_dist += weight_dist
    weight_dist = np.zeros((len(orders_df)), dtype=int)

orders_df["weight_dist"] = accumulated_weight_dist
orders_df

### Distance Matrix 산출


In [ ]:
def get_distance_matrix(mode, path=None):
    if mode == "LOAD":
        return pd.read_pickle(path)
    elif mode == "CREATE":
        routes_df = pd.DataFrame()

        distance_matrix = np.zeros(
            (len(orders_df) + WAREHOUSE_LEN, len(orders_df) + WAREHOUSE_LEN), dtype=int)

        for i in orders_df.index:
            for j in orders_df.index:
                distance_matrix[i][j] = np.ceil(np.sqrt(
                    (orders_df.loc[i, "pos_row"] -
                     orders_df.loc[j, "pos_row"]) ** 2 +
                    (orders_df.loc[i, "pos_col"] -
                     orders_df.loc[j, "pos_col"]) ** 2
                )).astype(int)
                distance_matrix[j][i] = distance_matrix[i][j]
        INF = int(1e7)
        for wh in warehouse_df.index:
            wh_row = warehouse_df.loc[wh, "pos_row"]
            wh_col = warehouse_df.loc[wh, "pos_col"]
            for order in orders_df.index:
                distance_matrix[len(orders_df) + wh][order] = np.ceil(np.sqrt(
                    (wh_row - orders_df.loc[order, "pos_row"]) ** 2 +
                    (wh_col - orders_df.loc[order, "pos_col"]) ** 2
                )).astype(int)
                distance_matrix[order][len(orders_df) + wh] = INF

        for wh_i in warehouse_df.index:
            for wh_j in warehouse_df.index:
                if wh_i == wh_j:
                    continue
                distance_matrix[len(orders_df) +
                                wh_i][len(orders_df)+wh_j] = INF
        with open(path, 'wb') as f:
            pickle.dump(distance_matrix, f)
        return distance_matrix


get_distance_matrix(mode="CREATE", path="./distance_matrix.pkl")

In [ ]:
distance_matrix = get_distance_matrix(
    mode="LOAD", path="./distance_matrix.pkl")
print(distance_matrix)


### Route_df 산출


In [ ]:
def get_route_df(mode, batch=1, path=None):
    if mode == "LOAD":
        return pd.read_pickle(path)
    elif mode == "CREATE":
        route_df = pd.DataFrame(
            columns=["wh_id", "route", "end_order", "distance"])
        for wh in warehouse_df.index:
            print(f"wh_id {wh} is processing...")
            # Make Index Mapping
            idx = list(np.where(from_wh_df == wh))
            idx_mapping = {i: item for i, item in enumerate(
                zip(idx[0], idx[1], idx[2]))}
            priority_df = orders_df.loc[idx[0], :]
            priority_df["index"] = [i for i in range(len(idx[0]))]

            priority_df = priority_df.sort_values(
                by=f"weight_dist_{wh}")

            for i in range(len(idx[0])):
                idx[0][i] = idx_mapping[priority_df.iloc[i]["index"]][0]
                idx[1][i] = idx_mapping[priority_df.iloc[i]["index"]][1]
                idx[2][i] = idx_mapping[priority_df.iloc[i]["index"]][2]

            # Batch Split
            batch_split = np.linspace(0, len(idx[0]), batch+1, dtype=int)

            for batch_idx in tqdm(range(batch), desc=f"batch progress"):
                batch_order_idx = idx[0][batch_split[batch_idx]:
                                         batch_split[batch_idx+1]]
                batch_product_idx = idx[1][batch_split[batch_idx]:
                                           batch_split[batch_idx+1]]
                batch_item_idx = idx[2][batch_split[batch_idx]:
                                        batch_split[batch_idx+1]]
                # Get wh_distance_matrix
                wh_distance_matrix = distance_matrix.copy()
                wh_distance_matrix[:len(orders_df), len(orders_df) + wh] = 0
                wh_distance_matrix = wh_distance_matrix[np.ix_(
                    np.append(batch_order_idx, len(orders_df) + wh),
                    np.append(batch_order_idx, len(orders_df) + wh))]

                # Get wh_demand_info
                mapper = {node: idx for node, idx in enumerate(
                    zip(batch_order_idx, batch_product_idx, batch_item_idx))}
                wh_demand_info = np.zeros(
                    (max(batch_order_idx) + 1, max(batch_product_idx) + 1, max(batch_item_idx + 1)), dtype=int)
                for order, product, item in zip(batch_order_idx, batch_product_idx, batch_item_idx):
                    wh_demand_info[order][product][item] = product_df.loc[product, "weight"]

                # Make Routing Model
                manager = pywrapcp.RoutingIndexManager(
                    len(wh_distance_matrix),
                    len(batch_order_idx),
                    len(wh_distance_matrix) - 1
                )

                routing = pywrapcp.RoutingModel(manager)

                def distance_callback(from_index, to_index):
                    from_node = manager.IndexToNode(from_index)
                    to_node = manager.IndexToNode(to_index)
                    return wh_distance_matrix[from_node][to_node]

                transit_callback_index = routing.RegisterTransitCallback(
                    distance_callback)
                routing.SetArcCostEvaluatorOfAllVehicles(
                    transit_callback_index)

                def demand_callback(from_index):
                    from_node = manager.IndexToNode(from_index)
                    if from_node == len(batch_order_idx):
                        return 0
                    else:
                        idx_tuple = mapper[from_node]
                        return wh_demand_info[idx_tuple[0]][idx_tuple[1]][idx_tuple[2]]
                        # return 200

                demand_callback_index = routing.RegisterUnaryTransitCallback(
                    demand_callback
                )
                routing.AddDimensionWithVehicleCapacity(
                    demand_callback_index,
                    0,
                    [PAYLOAD] * len(batch_order_idx),
                    True,
                    "Capacity"
                )

                search_parameters = pywrapcp.DefaultRoutingSearchParameters()
                search_parameters.first_solution_strategy = (
                    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

                solution = routing.SolveWithParameters(search_parameters)

                if solution:
                    for vehicle_id in range(len(batch_order_idx)):
                        index = routing.Start(vehicle_id)
                        route_distance = 0
                        route = []
                        while not routing.IsEnd(index):
                            node_index = manager.IndexToNode(index)
                            route.append(node_index)
                            previous_index = index
                            index = solution.Value(
                                routing.NextVar(index))
                            route_distance += routing.GetArcCostForVehicle(
                                previous_index, index, vehicle_id)
                        route.append(manager.IndexToNode(index))
                        if not route_distance:
                            continue
                        route = [mapper[x]
                                 for x in route if x != len(wh_distance_matrix) - 1]
                        route_df = pd.concat([route_df,
                                              pd.DataFrame({"wh_id": wh,
                                                            "route": [route],
                                                            "end_order": [route[-1]],
                                                            "distance": route_distance})],
                                             ignore_index=True
                                             )
                else:
                    raise Exception(
                        f"wh_id {wh}: routing could not be optimized")

        route_df.to_pickle(path)
        display(route_df)
        return route_df


route_df = get_route_df(mode="CREATE", batch=8, path="./route_df.pkl")


### 각 Route를 나타내는 command 산출 (Output File에 넣을 결과)


In [ ]:
def add_command_to_route(route_df):
    command_list = []
    for i in route_df.index:
        commands = []
        route = route_df.loc[i, "route"]
        wh_id = route_df.loc[i, "wh_id"]

        # Loading
        loading_dict = {}
        item_num_info = {}
        for order in route:
            if order[1] in loading_dict.keys():
                loading_dict[order[1]] += 1
            else:
                loading_dict[order[1]] = 1
            if (order[0], order[1]) in item_num_info.keys():
                item_num_info[(order[0], order[1])] += 1
            else:
                item_num_info[(order[0], order[1])] = 1
        for item, quantity in loading_dict.items():
            commands.append(f"L {wh_id} {item} {quantity}")

        # Delivering
        order_clear = np.full((len(orders_df), len(product_df)), False)

        for order in route:
            if not order_clear[order[0], order[1]]:
                commands.append(
                    f"D {order[0]} {order[1]} {item_num_info[(order[0], order[1])]}")
                order_clear[order[0], order[1]] = True

        command_list.append(commands)
    route_df["commands"] = command_list
    return route_df


route_df = add_command_to_route(route_df)
route_df

### Visualize Unit Routes Departing From Warehouse 5 and Warehouse 1


In [ ]:
cmap = mcm.get_cmap("nipy_spectral")
wh_colors = [cmap(i) for i in np.linspace(0, 1, num=WAREHOUSE_LEN)]

fig, ax = plt.subplots(figsize=(20, 8), nrows=1, ncols=2)
norm = mpl.colors.Normalize(vmin=0, vmax=WAREHOUSE_LEN-1)
cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
cb.set_label('Warehouse Index')

for i, wh in enumerate([5, 1]):
    ax[i].scatter(warehouse_df["pos_col"], warehouse_df["pos_row"],
                  ec="black", color=wh_colors, s=100, zorder=1)
    ax[i].scatter(orders_df["pos_col"], orders_df["pos_row"],
                  color="black", s=2, zorder=0)
    ax[i].set_xlabel("col")
    ax[i].set_ylabel("row")
    ax[i].axis("equal")

    for _, route in route_df.iterrows():
        if route["wh_id"] == wh:
            ax[i].plot([orders_df.loc[route["route"][0][0], "pos_col"], warehouse_df.loc[wh, "pos_col"]],
                       [orders_df.loc[route["route"][0][0], "pos_row"],
                           warehouse_df.loc[wh, "pos_row"]],
                       color="#29BCB7", lw=0.5)
            for prev_order, order in zip(route["route"][:-1], route["route"][1:]):
                ax[i].plot([orders_df.loc[prev_order[0], "pos_col"], orders_df.loc[order[0], "pos_col"]],
                           [orders_df.loc[prev_order[0], "pos_row"],
                               orders_df.loc[order[0], "pos_row"]],
                           color="#29BCB7", lw=0.5)

plt.show()

# 3. ⭐️ Scheduling ⭐️

### Problem

- 이제 우리는 방문해야 하는 Route 목록이 주어졌다.
- Route의 각 Route에 적절한 Drone을 적절한 시점에 배차해야 한다.
- 빨리 끝나는 작업이 빨리 처리되어야 한다. 이 점을 중심으로 하여 Drone을 적절하게 배차하시오.

# ⭐️ Solution ⭐️

- 단순화를 위해 Drone은 무조건 Warehouse에서 Load 하면 전부 다 배송을 마치고 다음 Warehouse로 이동한다고 가정하여 문제를 해결하자.
- Start Point -> End Point인 CVPR 문제로 치환하여 문제를 해결하고자 한다.
- 방문해야 하는 Route를 Node라고 생각한다.
- 이 때 Start Point와 Route 사이 거리는 각 Route에 대하여 ( 0번 창고 ~ Route의 시작 창고 위치까지 거리) + (Route의 총 distance) 이다.
- 각 Route 사이 거리는 (이전 Route의 마지막 주문 위치 ~ 다음 Route의 시작 창고 위치까지 거리) + (다음 Route의 총 distance) 이다.
- 각 Route에서 End Point 까지 거리는 0이다.
- Start Point에서 10개의 Drone이 모든 노드를 지나 End Point로 이동할 때 각 Drone 들의 경유 Node를 추적하면 된다.
- Route를 Batch 개로 나누어서 각 Batch에 해당하는 Route 끼리 계산한다.

  이 때 다음 Batch로 넘어갈 때는 이전 Batch에서 각 Drone의 마지막 경유 Node를 다음 Batch의 Start Point로 설정한다.

- Batch를 나눌 때 **빨리 처리할 수 있는 Order를 빠른 Batch에 넣어야 한다.**

## ⭐️⭐️

### 1. route의 distance는 작아야 한다.

### 2. route 내에 있는 Unit Order의 개수가 많다면 한 Drone이 한 번 Load하고 최대한 많은 주문을 처리한다는 소리가 된다. 이러한 Route를 먼저 배정해야 한다.

예를 들어 [(300, 1, 0)]로 1개의 Unit Order을 지나는 Route 보다 [(200, 1, 0), (201, 2, 0), (202, 3, 0), (202, 3, 1)] 를 먼저 처리하는 것이 더 좋다는 소리이다.

이는 간접적으로 weight 요소를 고려하게 된다.

### 3. 또 One_shot_order, 즉 한 Route 안에 하나의 Order set을 모두 처리할 수 있는 Route가 있다면 그 Route가 먼저 배정되는 것이 좋다.

예를 들어 400번 주문이 5, 6번 아이템을 요구하고, 401번 주문이 7, 8번 아이템을 요구한다고 가정하자.

그런데 이 주문을 처리하기 위한 Route가 다음과 같이 있다고 해보자.

1: [(400, 5, 0), (400, 6, 0)]

2: [(401, 7, 0)]

3: [(401, 8, 0)]

400번 주문은 Route 1를 이용하면 완전히 처리를 끝낼 수 있지만, 401번은 Route 2, 3 즉 2개의 Route를 이용해야 한다.

이 때 Route 1의 one_shot_order 의 개수는 1이다. (400번 주문)
Route 2, Route 3의 one_shot_order의 개수는 0이다.

### 이 1, 2, 3을 조합하여 아래와 같은 Priority Metric을 설정하였고 이 값이 작은 Route를 앞 Batch에서 먼저 처리한다.

$$
priority(route) = \frac{distance}{len(orders) \times (len(one \, shot \, order) + 1)}
$$

## ⭐️⭐️

### Output

- 배차 결과에 맞게 Command 값을 생성한다.
- 이를 `submission.csv`에 저장하여 제출한다.


## Drone Scheduling


In [ ]:
route_df

In [ ]:
def scheduling(batch=1, path=None):
    sol_commands = []
    INF = int(1e7)

    # Sorting Route_df
    priority = np.zeros((len(route_df)), dtype=int)
    for i, route in route_df.iterrows():
        one_shot_order_scaler = 1
        order_dict = {}
        for order in route["route"]:
            if order[0] not in order_dict.keys():
                order_dict[order[0]] = 1
            else:
                order_dict[order[0]] += 1
        for order, num in order_dict.items():
            if num == len(orders_df.loc[order, "ordered_items"]):
                one_shot_order_scaler += 1
        priority[i] = route["distance"] / \
            (len(route["route"]) * one_shot_order_scaler)
    route_df["priority"] = priority
    route_df.sort_values(by="priority", inplace=True, ignore_index=True)
#         route_df["route_len"] = route_df["route"].apply(lambda x: len(x))
#         route_df.sort_values(
#             by=["distance", "route_len"], ascending=[True, False], inplace=True, ignore_index=True)
    route_df.loc[:, "route_id"] = [i for i in range(len(route_df))]

    # Batch Split
    batch_split = np.linspace(0, len(route_df), batch+1, dtype=int)
    last_routes_id = []

    # Set Global Form of Starting Point
    start_loc = np.full(
        DRONE_NUM, -1, dtype=int)

    for batch_idx in tqdm(range(batch)):
        # print(f"batch {batch_idx} is processing...")
        batched_route_df = route_df.iloc[batch_split[batch_idx]:
                                         batch_split[batch_idx+1]]
        end_loc = np.full(
            DRONE_NUM, len(batched_route_df), dtype=int)

        # Transfer Start_loc Global Form to Local (Batch) Form
        min_start_point_idx = len(batched_route_df) + 1
        diff = 0
        start_route_id2idx = {}

        for i, loc in enumerate(start_loc):
            if loc in start_route_id2idx.keys():
                start_loc[i] = start_route_id2idx[loc]
            else:
                start_route_id2idx[loc] = min_start_point_idx + diff
                start_loc[i] = start_route_id2idx[loc]
                diff += 1

        start_idx2route_id = {v: k for k, v in start_route_id2idx.items()}

        batched_route_df.reset_index(inplace=True, drop=True)

        # Make Distance Matrix
        distance_matrix = np.full(
            ((len(batched_route_df) + 1 + diff), (len(batched_route_df) + 1 + diff)), INF, dtype=int)

        # Set zero on diagonal
        distance_matrix[np.diag_indices(distance_matrix.shape[0])] = 0

        # Distance from Start Points to Routes
        is_added = []
        for loc in start_loc:
            start_route_id = start_idx2route_id[loc]
            if start_route_id not in is_added:
                is_added.append(start_route_id)

                if start_route_id == -1:
                    for i in range(len(batched_route_df)):
                        distance_matrix[loc][i] = np.ceil(np.sqrt(
                            (warehouse_df.loc[batched_route_df.loc[i, "wh_id"], "pos_row"] -
                             warehouse_df.loc[0, "pos_row"]) ** 2 +
                            (warehouse_df.loc[batched_route_df.loc[i, "wh_id"], "pos_col"] -
                             warehouse_df.loc[0, "pos_col"]) ** 2
                        )).astype(int) + batched_route_df.loc[i, "distance"]
                else:
                    for i in range(len(batched_route_df)):
                        distance_matrix[loc][i] = np.ceil(np.sqrt(
                            (warehouse_df.loc[batched_route_df.loc[i, "wh_id"], "pos_row"] -
                             orders_df.loc[route_df.loc[start_route_id, "end_order"][0], "pos_row"]) ** 2 +
                            (warehouse_df.loc[batched_route_df.loc[i, "wh_id"], "pos_col"] -
                             orders_df.loc[route_df.loc[start_route_id, "end_order"][0], "pos_col"]) ** 2
                        )).astype(int) + batched_route_df.loc[i, "distance"]

        # Distance Between Routes
        for i in range(len(batched_route_df)):
            for j in range(len(batched_route_df)):
                if i == j:
                    continue
                distance_matrix[i][j] = np.ceil(np.sqrt(
                    (orders_df.loc[batched_route_df.loc[i, "end_order"][0], "pos_row"] -
                     warehouse_df.loc[batched_route_df.loc[j, "wh_id"], "pos_row"]) ** 2 +
                    (orders_df.loc[batched_route_df.loc[i, "end_order"][0], "pos_col"] -
                     warehouse_df.loc[batched_route_df.loc[j, "wh_id"], "pos_col"]) ** 2
                )).astype(int) + batched_route_df.loc[j, "distance"]

        # Set End Point
        distance_matrix[:, len(
            batched_route_df)] = 0
        # Make Routing Model
        manager = pywrapcp.RoutingIndexManager(
            len(distance_matrix),
            DRONE_NUM,
            start_loc.tolist(), end_loc.tolist()
        )

        routing = pywrapcp.RoutingModel(manager)

        # Set Distance Callback
        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return distance_matrix[from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(
            distance_callback
        )
        routing.SetArcCostEvaluatorOfAllVehicles(
            transit_callback_index
        )

        # Force routes to have roughly same length
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,
            INF,
            True,
            dimension_name
        )
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # set first solution heuristic
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        # search_parameters.time_limit.seconds = 10

        # Solve Routing Problem
        solution = routing.SolveWithParameters(search_parameters)

        if solution:
            # Get Solution Route
            last_routes_id = []
            for vehicle_id in range(DRONE_NUM):
                index = routing.Start(vehicle_id)
                route = []
                id_route = []
                route_distance = 0
                while not routing.IsEnd(index):
                    node_index = manager.IndexToNode(index)
                    route.append(node_index)
                    if node_index <= len(batched_route_df):
                        id_route.append(
                            batched_route_df.loc[node_index, "route_id"])
                    else:
                        id_route.append(start_idx2route_id[node_index])
                    previous_index = index
                    index = solution.Value(routing.NextVar(index))
                    route_distance += routing.GetArcCostForVehicle(
                        previous_index, index, vehicle_id)

                if len(id_route) > 1:
                    for route in id_route[1:]:
                        route_commands = route_df.loc[route, "commands"]
                        for command in route_commands:
                            sol_commands.append(f"{vehicle_id} " + command)

                last_routes_id.append(id_route[-1])

            start_loc = np.array(last_routes_id)

        else:
            raise Exception(
                f"batch {batch_idx}: routing could not be optimized")

    with open(path, 'w') as f:
        f.write("{}\n".format(len(sol_commands)))
        for command in sol_commands:
            f.write(command + "\n")
    return sol_commands


sol_commands = scheduling(batch=100, path="./submission.csv")

## Visualize Drone 0's Traveling Path


In [ ]:
cmap = mcm.get_cmap("nipy_spectral")
wh_colors = [cmap(i) for i in np.linspace(0, 1, num=WAREHOUSE_LEN)]

fig, ax = plt.subplots(figsize=(12, 8))
norm = mpl.colors.Normalize(vmin=0, vmax=WAREHOUSE_LEN-1)
cb = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
cb.set_label('Warehouse Index')

ax.scatter(warehouse_df["pos_col"], warehouse_df["pos_row"],
           ec="black", color=wh_colors, s=100, zorder=1)
ax.scatter(orders_df["pos_col"], orders_df["pos_row"],
           color="black", s=2, zorder=0)
ax.set_xlabel("col")
ax.set_ylabel("row")
ax.axis("equal")

drone_id = 0
prev_row = warehouse_df.loc[0, "pos_row"]
prev_col = warehouse_df.loc[0, "pos_col"]
for command in sol_commands:
    drone_num, _, order, _, _ = command.rstrip().split()
    if int(drone_num) == drone_id:
        row = orders_df.loc[int(order), "pos_row"]
        col = orders_df.loc[int(order), "pos_col"]
        ax.plot([prev_col, col], [prev_row, row], color="orangered", lw=0.5)
        prev_row = row
        prev_col = col

plt.show()